In [ ]:
#import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.tsaplots import plot_acf
import os
import scipy.optimize as optimize
import scipy.stats as stats

In [ ]:
''' Data Generation '''
################################'Generate X and Y'##############################
################################ Xt ~ N(0,1)
################################ Yt = α + βYt-1 + γXt + εt + θεt-1
def generate_data(T, alpha_y, beta_y, gamma, theta, sigma_y, mu, seed=None):
    np.random.seed(seed)
    df = pd.DataFrame(index=range(T), columns=['x', 'y', 'epsilon_y'])
    x = np.random.normal(0, 1, T)
    y1 = np.random.normal(mu, sigma_y)
    epsilon_y = sigma_y*np.random.randn(T)
    df.iloc[0,:] = [x[0], y1, epsilon_y[0]]
    
    for i in range(0, T-1):
        df.iloc[i+1,0] = x[i+1] 
        # Xt ~ N(0,1)
        df.iloc[i+1,1] = alpha_y + beta_y * df.iloc[i,1] + gamma * df.iloc[i+1,0] + epsilon_y[i+1] + theta * epsilon_y[i] # generate Y(t) recursively
        # Yt = α + βYt-1 + γXt + εt + θεt-1
        df.iloc[i+1,2] = epsilon_y[i+1]
    
    return df

df = generate_data(1000, 0.2, 0.5, 0.3, 0.9, 1, 0, seed=42)

In [ ]:
'''Gibbs Sampler'''

# simulate iteratively form distribution

# z_t^{-1} from a IG (Inverse Gaussian) ~ IG(3μ,λ) - mode μ[ np.sqrt(1 + 9μ**2/4λ**2) - 3μ/2λ] 
# beta ~ MVN(μ, Σ)                                    - mode m
# σ from Inverted Gamma ~ IΓ(α,β)                  - mode β/α +1

np.random.seed(123)
n = 1000
p = 3

# Define initial values
μ = 2
λ = 3
m = np.zeros(p)
Σ = np.eye(p)
print(m)
print(Σ)
α = 2
β = 3
z_1 = stats.invgauss.rvs(μ, scale=λ) #size=n
z = z_1.T
beta = np.random.multivariate_normal(m, Σ)
σ = stats.invgamma.rvs(α, scale=β)

x = df['x'].values
y = df['y'].values
y_lag = np.roll(y, 1) 
y_lag[0] = 0
X = np.column_stack((np.ones_like(x), y_lag, x))

print(beta)

p= 0.1
τ_2 = 2 / p * (1-p)
θ = (1 - 2*p) / p * (1-p)

In [ ]:
''' EM Algorithm 
def EM_Algorithm(df, M):

1. z_t^{-1} from a IG (Inverse Gaussian) ~ IG(3μ,λ) - mode μ[ np.sqrt(1 + 9μ**2/4λ**2) - 3μ/2λ] 
    initialize μ and λ based on the data (y) - mean
    
    for i in M = number of iterations
    # E step: compute the mode in closed form 
        mode = mu * (np.sqrt(1 + 9 * mu**2 / (4 * lambd**2)) - 3 * mu / (2 * lambd))
    
    # M step: update the parameters
        ?

    return mu, lambd
    ?how does it relate to z?
2. beta ~ MVN(μ, Σ) - mode m                                  
    initialize μ, Σ based on assumptions
    
    for i in M = number of iterations
    # E step: compute the mode in closed form 
        m
    
    # M step: update the parameters
        ?

    return m
    ?how does it relate to beta?

3. σ from Inverted Gamma ~ IΓ(α,β) - mode β/α +1
    initialize α,β based on assumptions
    
    for i in M = number of iterations
    # E step: compute the mode in closed form 
        β/α +1
    
    # M step: update the parameters
        ?

    return α,β
    ?how does it relate to sigma?

feed into liklihood 
'''

In [ ]:
import numpy as np
from scipy.stats import invgauss

def estimate_mode_parameters_inverse_gaussian(y, X, num_iterations=100):
    # Initialize the parameters of the Inverse Gaussian distribution
    mu = np.mean(y)
    lambd = 1 / np.var(y)
    
    for _ in range(num_iterations):
        # E-step: Compute the conditional mode
        mode = mu * (np.sqrt(1 + 9 * mu**2 / (4 * lambd**2)) - 3 * mu / (2 * lambd))
        
        # M-step: Update the parameters
        X_mode = X * np.reshape(mode, (-1,))  # Element-wise multiplication with reshaped mode
        XtX = X_mode.T @ X_mode
        XtX_inv = np.linalg.inv(XtX)
        XtX_inv_Xt = XtX_inv @ X_mode.T
        mu = XtX_inv_Xt @ y
        resid = y - X_mode @ mu
        lambd = np.sum(resid**2) / (X.shape[0] - X.shape[1])
        
    return mode, mu, lambd

# Example usage
y = df['y'].values.astype(float)  # Ensure dtype is float
x = df['x'].values.astype(float)  # Ensure dtype is float
y_lag = np.roll(y, 1)
y_lag[0] = 0
X = np.column_stack((np.ones_like(x), y_lag, x)).astype(float)  # Ensure dtype is float

mode, mu, lambd = estimate_mode_parameters_inverse_gaussian(y, X, num_iterations=1000)
print("Mode of the conditional distribution (Inverse Gaussian):", mode)
print("Updated parameters - mu:", mu)
print("Updated parameters - lambda:", lambd)


In [ ]:
''' Inverse Gaussian for z'''

def estimate_mode_parameters_inverse_gaussian(y, x, num_iterations=1000):
    # Initialize the parameters of the Inverse Gaussian distribution
    mu = np.mean(y)
    lambd = 1 / np.var(y)
    
    for _ in range(num_iterations):
        # E-step: Compute the conditional mode
        mode = mu * (np.sqrt(1 + 9 * mu**2 / (4 * lambd**2)) - 3 * mu / (2 * lambd))
        
        # M-step: Update the parameters
        X_mode = x * mode  # Element-wise multiplication with reshaped mode
        XtX = X_mode.T @ X_mode
        XtX_inv = np.linalg.inv(XtX)
        XtX_inv_Xt = XtX_inv @ X_mode.T
        mu = XtX_inv_Xt @ y
        resid = y - X_mode @ mu
        lambd = np.sum(resid**2) / (X.shape[0] - X.shape[1])
        
    return mode, mu, lambd

# Example usage
y = df['y'].values.astype(float)  # Ensure dtype is float
x = df['x'].values.astype(float)  # Ensure dtype is float

mode, mu, lambd = estimate_mode_parameters_inverse_gaussian(y, X, num_iterations=1000)
print("Mode of the conditional distribution (Inverse Gaussian):", mode)
print("Updated parameters - mu:", mu)
print("Updated parameters - lambda:", lambd)

In [ ]:
import numpy as np
from scipy.stats import multivariate_normal

def estimate_mode_multivariate_normal(data, num_iterations=100):
    # Initialize the parameters of the Multivariate Normal distribution
    mean = np.mean(data, axis=0)
    cov = np.cov(data, rowvar=False)
    
    for _ in range(num_iterations):
        # E-step: Compute the conditional mode
        mode = mean
        
        # M-step: Update the parameters
        mean = np.mean(data, axis=0)
        cov = np.cov(data, rowvar=False)
        
    return mode

# Example usage
data = np.array([[1.2, 0.8], [1.4, 1.0], [1.6, 1.2], [1.8, 1.4], [2.0, 1.6]])  # Input data
mode = estimate_mode_multivariate_normal(data)
print("Mode of the conditional distribution (Multivariate Normal):", mode)


In [ ]:
import numpy as np
from scipy.stats import invgamma

def estimate_mode_inverted_gamma(data, num_iterations=100):
    # Initialize the parameters of the Inverted Gamma distribution
    alpha = 2
    beta = 3
    
    for _ in range(num_iterations):
        # E-step: Compute the conditional mode
        mode = beta / alpha + 1
        
        # M-step: Update the parameters
        alpha = alpha + 1
        beta = np.mean(data) + beta
        
    return mode

# Example usage
data = np.array([0.8, 1.2, 1.5, 1.0, 1.3])  # Input data
mode = estimate_mode_inverted_gamma(data)
print("Mode of the conditional distribution (Inverted Gamma):", mode)


In [ ]:
import numpy as np
from scipy.stats import invgamma

def estimate_mode_parameters_inverted_gamma(y, X, num_iterations=100):
    # Initialize the parameters of the Inverted Gamma distribution
    alpha = 2
    beta = 3
    
    for _ in range(num_iterations):
        # E-step: Compute the conditional mode
        mode = beta / alpha + 1
        
        # M-step: Update the parameters
        alpha = alpha + 1
        beta = np.mean(y) + beta
        
    return mode, alpha, beta

